In [ ]:
!pip install neo4j neo4j-graphrag -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.


In [ ]:
from neo4j import GraphDatabase
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [ ]:
def load_properties(path):
    data = {}
    with open(path) as f:
        for line in f:
            if "=" in line:
                k, v = line.split("=", 1)
                data[k.strip()] = v.strip()
    return data

In [5]:
import os
props = load_properties("openai_key.txt")

os.environ["OPENAI_KEY"] = props["OPENAI_KEY"]
os.environ["NEO4J_URI"] = props["NEO4J_URI"]
os.environ["NEO4J_USERNAME"] = props["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"] = props["NEO4J_PASSWORD"]
#os.environ["NEO4J_DATABASE"] = props["NEO4J_DATABASE"]


In [29]:
neo4j_driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"),
    auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
)

In [30]:
neo4j_driver.verify_connectivity()

In [31]:
llm = OpenAILLM(
    model_name="gpt-4o",
    api_key=os.environ["OPENAI_KEY"],
    model_params={
        "temperature": 0,
        "response_format": {"type": "json_object"},
    }
)

In [32]:
embedder = OpenAIEmbeddings(
    api_key=os.environ["OPENAI_KEY"],
    model="text-embedding-ada-002"
    )

In [23]:
#without text splitter
'''
kg_builder = SimpleKGPipeline(
    llm=llm,
    driver=neo4j_driver,
    neo4j_database=os.getenv("NEO4J_DATABASE"),
    embedder=embedder,
    from_pdf=True,
)

In [34]:
#using text splitter for creating data chunks
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter

text_splitter = FixedSizeSplitter(chunk_size=500, chunk_overlap=50)

In [37]:
#Define schema - what nodes types and relationships I am interested to extract
NODE_TYPES = [
    "Technology",
    "Concept",
    "Example",
    "Process",
    "Challenge",
    {"label": "Benefit", "description": "A benefit or advantage of using a technology or approach."},
    {
        "label": "Resource",
        "description": "A related learning resource such as a book, article, video, or course.",
        "properties": [
            {"name": "name", "type": "STRING", "required": True},
            {"name": "type", "type": "STRING"}
        ]
    },
]

RELATIONSHIP_TYPES = [
    "RELATED_TO",
    "PART_OF",
    "USED_IN",
    "LEADS_TO",
    "HAS_CHALLENGE",
    "LEADS_TO",
    "CITES"
]

PATTERNS = [
    ("Technology", "RELATED_TO", "Technology"),
    ("Concept", "RELATED_TO", "Technology"),
    ("Example", "USED_IN", "Technology"),
    ("Process", "PART_OF", "Technology"),
    ("Technology", "HAS_CHALLENGE", "Challenge"),
    ("Concept", "HAS_CHALLENGE", "Challenge"),
    ("Technology", "LEADS_TO", "Benefit"),
    ("Process", "LEADS_TO", "Benefit"),
    ("Resource", "CITES", "Technology"),
]


In [38]:
kg_builder = SimpleKGPipeline(
    llm=llm,
    driver=neo4j_driver,
    neo4j_database=os.getenv("NEO4J_DATABASE"),
    embedder=embedder,
    from_pdf=True,
    text_splitter=text_splitter,
     schema={
        "node_types": NODE_TYPES,
        "relationship_types": RELATIONSHIP_TYPES,
        "patterns": PATTERNS
    },
)

In [39]:
pdf_file = "genai-fundamentals_4-integrating-neo4j_6-frameworks.pdf"
result = await kg_builder.run_async(file_path=pdf_file)
print(result)

run_id='f07dcbe5-6d92-443c-bca9-e9c81cfde136' result={'resolver': {'number_of_nodes_to_resolve': 43, 'number_of_created_nodes': 35}}


##The unstructured part of your graph is known as the **Lexical Graph**
##The structured part is known as the **Domain Graph**

In [ ]:
#Loading structured data to the knowledge Graph

In [54]:
import csv

csv_file = "StructuredData/docs.csv"

#docs_csv = csv.DictReader(
#    open(os.path.join(data_path, "docs.csv"), encoding="utf8", newline='')
#)

# Open the CSV file
with open(csv_file, newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)  # Reads each row as a dictionary
    for row in reader:
        print(row)  # Each row is a dict: {'filename': 'doc1.pdf', 'title': 'Intro', ...}
        # Access columns
        print(row['filename'], row['course'], row['module'],row['lesson'],row['url'])
        pdf_path=row['filename']
        url=row['url']
        lesson=row['lesson']
        course=row['course']
        module=row['module']




{'filename': 'genai-fundamentals_4-integrating-neo4j_6-frameworks.pdf', 'course': 'genai-fundamentals', 'module': '4-integrating-neo4j', 'lesson': '6-frameworks', 'url': 'https://graphacademy.neo4j.com/courses/genai-fundamentals/4-integrating-neo4j/6-frameworks'}
genai-fundamentals_4-integrating-neo4j_6-frameworks.pdf genai-fundamentals 4-integrating-neo4j 6-frameworks https://graphacademy.neo4j.com/courses/genai-fundamentals/4-integrating-neo4j/6-frameworks


In [55]:
cypher = """
MATCH (d:Document {path: $pdf_path})
MERGE (l:Lesson {url: $url})
SET l.name = $lesson,
    l.module = $module,
    l.course = $course
MERGE (d)-[:PDF_OF]->(l)
"""


In [56]:
 # Create structured graph

params = {
    "pdf_path": pdf_path,
    "url": url,
    "lesson": lesson,
    "module":module ,
    "course": course
}

records, summary, keys = neo4j_driver.execute_query(
    cypher,
    parameters_=params,
    database_=os.getenv("NEO4J_DATABASE")
)